In [38]:
from pathlib import Path
import json
import pandas as pd

# app_dir = Path(__file__).parent

# EXCEL IMPORTS
voted_df = "data/2025/2025.json";

# FUNCTION
def wrangle_voted():
    # Load JSON file
    with open(r"data/2025/2025.json", "r", encoding="utf-8") as f:
        data = json.load(f)

    # Flatten the nested JSON
    # data.values() returns a list of lists (one per district)
    all_records = []
    for district_list in data.values():
        all_records.extend(district_list)

    df = pd.DataFrame(all_records)
    df["votes"] = pd.to_numeric(df["votes"], errors="coerce").round()
    df['district'] = df['district'].str.strip()
    df['district'] = df['district'].replace('Mzuzu City', 'Mzimba')
    df['district'] = df['district'].replace('Blantyre City', 'Blantyre')
    df['district'] = df['district'].replace('Karonga Town', 'Karonga')
    df['district'] = df['district'].replace('Kasungu Municipality', 'Kasungu')
    df['district'] = df['district'].replace('Lilongwe Citv', 'Lilongwe')
    df['district'] = df['district'].replace('Lilongwe Rural', 'Lilongwe')
    df['district'] = df['district'].replace('Luchenza Municipality', 'Thyolo')
    df['district'] = df['district'].replace('Mangochi Municipality', 'Mangochi')
    df['district'] = df['district'].replace('Zomba City', 'Zomba')

    df['Region'] = "";

    # Mapping Regions to districts
    districts_regions = {'Mwanza': 'Southern', 'Ntcheu': 'Central', 'Neno': 'Southern', 'Nkhata Bay': 'Northern', 'Chikwawa': 'Southern', 
                         'Nsanje': 'Southern', 'Nkhotakota': 'Central', 'Salima': 'Central', 'Mangochi': 'Southern', 'Machinga': 'Southern', 
                         'Dedza': 'Central', 'Karonga': 'Northern', 'Chitipa': 'Northern', 'Balaka': 'Southern', 'Zomba': 'Southern', 
                         'Mchinji': 'Central', 'Blantyre': 'Southern', 'Mzimba': 'Northern', 'Likoma': 'Northern', 'Rumphi': 'Northern', 
                         'Kasungu': 'Central', 'Mulanje': 'Southern', 'Thyolo': 'Southern', 'Chiradzulu': 'Southern', 'Lilongwe': 'Central', 
                         'Ntchisi': 'Central', 'Phalombe': 'Southern', 'Dowa': 'Central'}
    df['Region'] = df['district'].map(districts_regions)
    return df

_2025_df_final = wrangle_voted()
_2025_district_summary = pd.pivot_table(_2025_df_final, index=['district', 'Region'], values=['votes'], aggfunc='sum')

def wrangle_registered():
    registered_df = pd.read_excel("data/2025/Registered_2025.xlsx")
    registered_df = registered_df.rename(columns={'District': 'district'})
    registered_df['district'] = registered_df['district'].str.strip()
    registered_df['district'] = registered_df['district'].replace('Mzuzu City', 'Mzimba')
    registered_df['district'] = registered_df['district'].replace('M\'mbelwa', 'Mzimba')
    registered_df['district'] = registered_df['district'].replace('Blantyre City', 'Blantyre')
    registered_df['district'] = registered_df['district'].replace('Karonga Town', 'Karonga')
    registered_df['district'] = registered_df['district'].replace('Kasungu Municipality', 'Kasungu')
    registered_df['district'] = registered_df['district'].replace('Lilongwe City', 'Lilongwe')
    registered_df['district'] = registered_df['district'].replace('Lilongwe Rural', 'Lilongwe')
    registered_df['district'] = registered_df['district'].replace('Luchenza Municipality', 'Thyolo')
    registered_df['district'] = registered_df['district'].replace('Mangochi Municipality', 'Mangochi')
    registered_df['district'] = registered_df['district'].replace('Zomba City', 'Zomba')
    # registered_df['district'] = registered_df['district'].replace('Nkhata Bay', 'Nkhatabay')
    registered_df["Total"] = pd.to_numeric(registered_df["Total"], errors="coerce").round()
    return registered_df

registered = wrangle_registered()
registered_summary = pd.pivot_table(registered, index=['district'], values=['Total'], aggfunc='sum')


registered_voted = pd.merge(_2025_district_summary, registered_summary, on="district", how="outer")
registered_voted = registered_voted.rename(columns={'Total': 'Number Of Registred Voters 25', 'votes': 'Total Number Voted 25'})
registered_voted['Voter_Empathy 25'] = ((registered_voted['Number Of Registred Voters 25'] - registered_voted['Total Number Voted 25']) / registered_voted['Number Of Registred Voters 25']) * 100
registered_voted['Voter_Cast 25'] = ((registered_voted['Total Number Voted 25'])) * 100
registered_voted['Voter_Turnout 25'] = (((registered_voted['Total Number Voted 25']) / registered_voted['Number Of Registred Voters 25']) * 100).round()


# _2025_df_final.to_excel("data/2025/2025.xlsx", index=False)
MEC_2025 = registered_voted.sort_values(by="Voter_Turnout 25", ascending=True)
MEC_2025.head()


,Total Number Voted 25,Number Of Registred Voters 25,Voter_Empathy 25,Voter_Cast 25,Voter_Turnout 25
district,,,,,
Nkhata Bay,70808.0,107406,34.074446,7080800.0,66.0
Salima,123115.0,186585,34.016668,12311500.0,66.0
Nkhotakota,113062.0,172331,34.392535,11306200.0,66.0
Kasungu,246994.0,370686,33.368403,24699400.0,67.0
Karonga,99324.0,149132,33.398600,9932400.0,67.0


In [11]:
from pathlib import Path

import pandas as pd

# EXCEL IMPORTS
xlsx_df = "data/2020-Fresh-Presidential-Election-Results-Per-station.xlsx";

# FUNCTION
def wrangle_luanar(path):
    df = pd.read_excel(path)
    df['District Name'] = df['District Name'].fillna(method='ffill')
    df['Constituency Name'] = df['Constituency Name'].fillna(method='ffill')
    df['Ward Name'] = df['Ward Name'].fillna(method='ffill')
    df['Centre Code'] = df['Centre Code'].fillna(method='ffill')
    df['Centre Name'] = df['Centre Name'].fillna(method='ffill')
    df['District Name'] = df['District Name'].replace('(Thekerani)', 'Thyolo')
    df['District Name'] = df['District Name'].replace('Chikhwawa', 'Chikwawa')
    df['Region'] = "";

    # Mapping Regions to districts
    districts_regions = {'Mwanza': 'Southern', 'Ntcheu': 'Central', 'Neno': 'Southern', 'Nkhatabay': 'Northern', 'Chikwawa': 'Southern', 
                         'Nsanje': 'Southern', 'Nkhotakota': 'Central', 'Salima': 'Central', 'Mangochi': 'Southern', 'Machinga': 'Southern', 
                         'Dedza': 'Central', 'Karonga': 'Northern', 'Chitipa': 'Northern', 'Balaka': 'Southern', 'Zomba': 'Southern', 
                         'Mchinji': 'Central', 'Blantyre': 'Southern', 'Mzimba': 'Northern', 'Likoma': 'Northern', 'Rumphi': 'Northern', 
                         'Kasungu': 'Central', 'Mulanje': 'Southern', 'Thyolo': 'Southern', 'Chiradzulu': 'Southern', 'Lilongwe': 'Central', 
                         'Ntchisi': 'Central', 'Phalombe': 'Southern', 'Dowa': 'Central'}
    df['Region'] = df['District Name'].map(districts_regions)

    df = df.drop(columns='Unnamed: 2')
    df = df[df['Centre Code'] != 'Total']
    df['Number Of Registred Voters'] = df['Number Of Registred Voters'].mask(df.duplicated(subset='Centre Code', keep='first'), 0)

    return df

_1_2020_df_final = wrangle_luanar(xlsx_df)
# District Summary
district_summary = pd.pivot_table(_1_2020_df_final, index=['District Name', 'Region'], values=['Number Of Registred Voters', 'Total Number Voted'], aggfunc='sum')
district_summary['Voter_Empathy'] = ((district_summary['Number Of Registred Voters'] - district_summary['Total Number Voted']) / district_summary['Number Of Registred Voters']) * 100
district_summary['Voter_Cast'] = ((district_summary['Total Number Voted'])) * 100
district_summary['Voter_Turnout'] = (((district_summary['Total Number Voted']) / district_summary['Number Of Registred Voters']) * 100)
district_summary = district_summary.reset_index()
district_summary = district_summary.rename(columns={'District Name': 'District_Name', 'Number Of Registred Voters' : 'Number_Of_Registred_Voters', 'Total Number Voted': 'Total_Number_Voted'})
district_summary["District_Name"] = district_summary["District_Name"].astype(str)
district_summary["Voter_Empathy"] = pd.to_numeric(district_summary["Voter_Empathy"], errors="coerce").round()
district_summary["Voter_Turnout"] = pd.to_numeric(district_summary["Voter_Turnout"], errors="coerce").round()
district_summary["Voter_Cast"] = pd.to_numeric(district_summary["Voter_Cast"], errors="coerce").round()
district_summary = district_summary.sort_values(by='Voter_Empathy', ascending=False)
district_summary['Longtude'] = "";
district_summary['Latitude'] = "";

# Mapping latitude to districts
districts_longtudes = {'Blantyre': '35.0058', 'Lilongwe': '33.7833', 'Mzuzu': '34.0151', 'Zomba': '35.3192', 'Karonga': '33.9333', 
                           'Kasungu': '33.4833', 'Mangochi': '35.2667', 'Salima': '34.4333', 'Likoma': '34.7333', 'Dedza': '34.3333', 
                           'Nkhotakota': '34.3', 'Mchinji': '32.9', 'Nsanje': '35.2667', 'Mzimba': '33.6', 'Rumphi': '33.8667', 
                           'Ntcheu': '34.6667', 'Mulanje': '35.5081', 'Mwanza': '34.5178', 'Chitipa': '33.27', 'Nkhatabay': '34.3', 
                           'Ntchisi': '34', 'Dowa': '33.9167', 'Thyolo': '35.1333', 'Phalombe': '35.6533', 'Chiradzulu': '35.1833', 
                           'Machinga': '35.5167', 'Balaka': '34.9591', 'Neno': '34.6534', 'Chikwawa': '34.801'}
district_summary['Longtude'] = district_summary['District_Name'].map(districts_longtudes)

# Mapping longtude to districts
districts_latitudes = {
        'Blantyre': '-15.7861', 'Lilongwe': '-13.9833', 'Mzuzu': '-11.4581', 'Zomba': '-15.3869', 'Karonga': '-9.9333', 'Kasungu': '-13.0333', 
        'Mangochi': '-14.4667', 'Salima': '-13.7833', 'Likoma': '-12.0667', 'Dedza': '-14.3667', 'Nkhotakota': '-12.9167', 
        'Mchinji': '-13.8167', 'Nsanje': '-16.9167', 'Mzimba': '-11.9', 'Rumphi': '-11.0167', 'Ntcheu': '-14.8333', 'Mulanje': '-16.0258', 
        'Mwanza': '-15.5986', 'Chitipa': '-9.7019', 'Nkhatabay': '-11.6', 'Ntchisi': '-13.3667', 'Dowa': '-13.6667', 'Thyolo': '-16.0667', 
        'Phalombe': '-15.8033', 'Chiradzulu': '-15.7', 'Machinga': '-14.9667', 'Balaka': '-14.9889', 'Neno': '-15.3981', 'Chikwawa': '-16.035'}
district_summary['Latitude'] = district_summary['District_Name'].map(districts_latitudes)

# Region Summary
region_summary = pd.pivot_table(_1_2020_df_final, index=['Region'], values=['Number Of Registred Voters', 'Total Number Voted'], aggfunc='sum')
region_summary['Voter_Empathy'] = ((region_summary['Number Of Registred Voters'] - region_summary['Total Number Voted']) / region_summary['Number Of Registred Voters']) * 100
region_summary = region_summary.reset_index()
region_summary = region_summary.rename(columns={'Number Of Registred Voters' : 'Number_Of_Registred_Voters', 'Total Number Voted': 'Total_Number_Voted'})
region_summary = region_summary.sort_values(by='Voter_Empathy', ascending=False)

district_summary.head()
# _1_2020_df_final.to_excel(export_path + "first_cleaned_all.xlsx", index=False)
# district_summary.to_excel(export_path + "summary.xlsx", index=True)

C:\Users\Francis\AppData\Local\Temp\ipykernel_11916\1643978945.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['District Name'] = df['District Name'].fillna(method='ffill')
C:\Users\Francis\AppData\Local\Temp\ipykernel_11916\1643978945.py:12: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Constituency Name'] = df['Constituency Name'].fillna(method='ffill')
C:\Users\Francis\AppData\Local\Temp\ipykernel_11916\1643978945.py:13: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Ward Name'] = df['Ward Name'].fillna(method='ffill')
C:\Users\Francis\AppData\Local\Temp\ipykernel_11916\1643978945.py:14: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfil

,District_Name,Region,Number_Of_Registred_Voters,Total_Number_Voted,Voter_Empathy,Voter_Cast,Voter_Turnout,Longtude,Latitude
15,Mwanza,Southern,48307.0,22157.0,54.0,2215700.0,46.0,34.5178,-15.5986
21,Ntcheu,Central,228136.0,111559.0,51.0,11155900.0,49.0,34.6667,-14.8333
17,Neno,Southern,55857.0,29281.0,48.0,2928100.0,52.0,34.6534,-15.3981
2,Chikwawa,Southern,257547.0,141998.0,45.0,14199800.0,55.0,34.801,-16.035
18,Nkhatabay,Northern,106374.0,58450.0,45.0,5845000.0,55.0,34.3,-11.6


In [43]:
# MEC_2025 = MEC_2025.reset_index().rename(columns={'district': 'District_Name'})

consolidated = pd.merge(MEC_2025, district_summary, on="District_Name", how="inner")

consolidated = consolidated[['District_Name', 'Voter_Turnout', 'Voter_Turnout 25']]
consolidated = consolidated.rename(columns={'Voter_Turnout': '2020', 'Voter_Turnout 25': '2025'})

consolidated.head(50)



,District_Name,2020,2025
0,Salima,59.0,66.0
1,Nkhotakota,57.0,66.0
2,Kasungu,69.0,67.0
3,Karonga,61.0,67.0
4,Mchinji,63.0,68.0
5,Dedza,61.0,69.0
6,Rumphi,68.0,70.0
7,Dowa,76.0,71.0
8,Mzimba,67.0,71.0
9,Lilongwe,71.0,72.0


In [49]:
import plotly.express as px

# Flag where 2025 < 2020
consolidated["Flag"] = consolidated["2025"] < consolidated["2020"]

# Melt data for easier plotting
df_melt = consolidated.melt(id_vars=["District_Name", "Flag"], 
                            value_vars=["2020", "2025"], 
                            var_name="Year", 
                            value_name="Turnout")

# Create plot
fig = px.bar(
    df_melt,
    x="District_Name",
    y="Turnout",
    color="Year",
    barmode="group",
    text="Turnout",
    title="Comparison of Voter Turnout (2020 vs 2025)"
)

# Highlight flagged districts (2025 lower than 2020)
for i, row in consolidated.iterrows():
    if row["Flag"]:
        fig.add_annotation(
            x=row["District_Name"],
            y=row["2025"],
            text="↓",
            showarrow=False,
            font=dict(color="red", size=20)
        )

fig.update_traces(texttemplate='%{text:.1f}', textposition="outside")
fig.update_layout(yaxis_title="Turnout (%)")

fig.show()


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [48]:
pip install nbformat


   ---------------------------------------- 0/7 [fastjsonschema]
   ----- ---------------------------------- 1/7 [rpds-py]
   ----------- ---------------------------- 2/7 [attrs]
   ----------- ---------------------------- 2/7 [attrs]
   ----------- ---------------------------- 2/7 [attrs]
   ----------------- ---------------------- 3/7 [referencing]
   ----------------- ---------------------- 3/7 [referencing]
   ----------------- ---------------------- 3/7 [referencing]
   ---------------------------- ----------- 5/7 [jsonschema]
   ---------------------------- ----------- 5/7 [jsonschema]
   ---------------------------- ----------- 5/7 [jsonschema]
   ---------------------------- ----------- 5/7 [jsonschema]
   ---------------------------- ----------- 5/7 [jsonschema]
   ---------------------------- ----------- 5/7 [jsonschema]
   ---------------------------------- ----- 6/7 [nbformat]
   ---------------------------------- ----- 6/7 [nbformat]
   ---------------------------------- 